In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.ITU8J2_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
import tensorflow as tf

In [ ]:
# Dividir em treino e teste manualmente
train_size = int(0.8 * info.splits['train'].num_examples)  # 80% para treino
test_size = info.splits['train'].num_examples - train_size  # 20% para teste

# Dividir em dois conjuntos
train_dataset = dataset['train'].take(train_size)
test_dataset = dataset['train'].skip(train_size)

# Pré-processar os dados
train_dataset = train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

base_model.trainable = False


<ipython-input-9-0bca1ad5c6aa>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=5, validation_data=test_dataset)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 100ms/step - accuracy: 0.9277 - loss: 0.1734 - val_accuracy: 0.9648 - val_loss: 0.0954
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 87s 118ms/step - accuracy: 0.9667 - loss: 0.0842 - val_accuracy: 0.9652 - val_loss: 0.0909
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 117ms/step - accuracy: 0.9702 - loss: 0.0758 - val_accuracy: 0.9648 - val_loss: 0.0892
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 69s 118ms/step - accuracy: 0.9721 - loss: 0.0706 - val_accuracy: 0.9658 - val_loss: 0.0885
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 63s 85ms/step - accuracy: 0.9733 - loss: 0.0669 - val_accuracy: 0.9663 - val_loss: 0.0885


In [ ]:
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(train_dataset, epochs=5, validation_data=test_dataset)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 97s 132ms/step - accuracy: 0.9416 - loss: 0.1420 - val_accuracy: 0.9684 - val_loss: 0.1251
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 69s 119ms/step - accuracy: 0.9938 - loss: 0.0195 - val_accuracy: 0.9723 - val_loss: 0.1009
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 51s 88ms/step - accuracy: 0.9998 - loss: 0.0029 - val_accuracy: 0.9766 - val_loss: 0.0892
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 69s 119ms/step - accuracy: 0.9999 - loss: 8.5250e-04 - val_accuracy: 0.9768 - val_loss: 0.0897
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 62s 85ms/step - accuracy: 1.0000 - loss: 2.7204e-04 - val_accuracy: 0.9757 - val_loss: 0.0929


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test accuracy: {test_acc}')

146/146 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.9760 - loss: 0.0948
Test accuracy: 0.9757145643234253
